# Kannada challenge

[Link to the challenge](https://www.kaggle.com/c/Kannada-MNIST/overview)

Wide Residual Network: [Github](https://github.com/EricAlcaide/keras-wrn)

### Imports

In [26]:
import pandas as pd
import numpy as np
import csv
import seaborn as sn
import matplotlib.pyplot as plt
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Dense, Activation, ZeroPadding2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
from tensorflow.python.client import device_lib
import keras.backend as K

import wrn as wrn

from tensorflow.python.client import device_lib

In [27]:
#read train.csv
train_dataset1 = pd.read_csv('train.csv')
train_dataset2 = pd.read_csv('Dig-MNIST.csv')
train_dataset = train_dataset1.append(train_dataset2, ignore_index = True)

test_dataset = pd.read_csv('test.csv')

In [28]:
#Shuffle train_dataaset
train_dataset = train_dataset.sample(frac=1).reset_index(drop=True)
#train_dataset = shuffle(train_dataset).reset_index(drop=True) # other way to do the same
train_dataset.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
X = train_dataset.drop(['label'], axis=1).values.reshape(-1, 28, 28, 1).astype('float32')/255
y = train_dataset["label"]
x_test = test_dataset.drop(["id"], axis=1).values.reshape(-1, 28, 28, 1).astype('float32')/255
X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.1)

In [30]:
y_train = np_utils.to_categorical(y_train, num_classes=10)
y_dev = np_utils.to_categorical(y_dev, num_classes=10)

In [31]:
#ip = Input(shape=(1, 28, 28))
init_shape = (1, 28, 28) if K.image_dim_ordering() == 'th' else (28, 28, 1)
#Model WRN-16-8
model = wrn.build_model(init_shape, 10, 16, 8, dropout=0)

In [32]:
model.compile("adam", "categorical_crossentropy", ["accuracy"])

In [33]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_dev, y_dev), verbose=1)

Train on 63216 samples, validate on 7024 samples
Epoch 1/10
63216/63216 [==============================] - 199s 3ms/step - loss: 0.1064 - acc: 0.9685 - val_loss: 0.0941 - val_acc: 0.9691
Epoch 2/10
63216/63216 [==============================] - 198s 3ms/step - loss: 0.0502 - acc: 0.9849 - val_loss: 0.0439 - val_acc: 0.9880
Epoch 3/10
63216/63216 [==============================] - 195s 3ms/step - loss: 0.0386 - acc: 0.9882 - val_loss: 0.0487 - val_acc: 0.9852
Epoch 4/10
63216/63216 [==============================] - 197s 3ms/step - loss: 0.0314 - acc: 0.9903 - val_loss: 0.0852 - val_acc: 0.9744
Epoch 5/10
63216/63216 [==============================] - 202s 3ms/step - loss: 0.0242 - acc: 0.9925 - val_loss: 0.0609 - val_acc: 0.9819
Epoch 6/10
63216/63216 [==============================] - 204s 3ms/step - loss: 0.0207 - acc: 0.9935 - val_loss: 0.0351 - val_acc: 0.9900
Epoch 7/10
63216/63216 [==============================] - 209s 3ms/step - loss: 0.0165 - acc: 0.9947 - val_loss: 0.0359 - v

In [34]:
#predicts
predictions = model.predict(X_dev)
predictions = np.argmax(predictions, axis=1)

In [35]:
y_dev = np.argmax(y_dev, axis=1)

In [36]:
correct_predictions = 0
for i in range(7024):
    if(predictions[i] == y_dev[i]):
        correct_predictions+=1
print("Final accuracy over test:", correct_predictions/7024)

Final accuracy over test: 0.9923120728929385
